## Imports and Data

In [ ]:
# PLOTTING
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import seaborn as sns
import pandas as pd
import calmap
import datetime

In [ ]:
df = pd.read_csv('data/cacas_latest.csv')
dateString = datetime.datetime.now().strftime("%d-%m")

## Poop Ranking

In [ ]:
# Group by 'user' and calculate the sum for each group
result_df = df.groupby('user').agg({
    'caca': 'sum',
    'caca en el trabajo': 'sum'
}).reset_index()

# Rename columns for clarity
result_df.columns = ['competidor', 'Caca', 'Caca Laboral']
# 
# melt the dataframe to long format
result_df = result_df.melt(id_vars='competidor', var_name='tipo', value_name='cantidad')
# sort the dataframe by 'cantidad'
result_df.sort_values('cantidad', ascending=False, inplace=True)
result_df

In [ ]:
# Plot the data
plt.figure(figsize=(10, 6))
# sns.catplot(data=result_df, x='competidor', y='cantidad', hue='tipo', kind='bar', height=6, aspect=2)

# Create the bar plot
sns.barplot(
    data=result_df,
    x="competidor",
    y="cantidad",
    hue="tipo",
    # Style
    palette="YlOrBr",
    width=.8, 
    linewidth=2.5,
)
# 
# Customize labels and title
plt.xlabel("Competidor", fontsize=14)
plt.ylabel("Cantidad de Cacas", fontsize=14)
plt.title("Ranking Gran Cagatón Anual", fontsize=16)

# Move legend to top right
plt.legend(title="Tipo de Caca", loc="upper right", bbox_to_anchor=(1.25, 1))  # Adjust x-position if needed

# Rotate x-axis labels for better readability
plt.xticks(rotation=45, ha="right")

# Show the plot
plt.tight_layout()

# save the plot in the 'plots' folder with day and month in the filename
plt.savefig(f'plots/ranking_{dateString}.png')

plt.show()

## Poop Calendar

In [ ]:
# melt into long format, having "caca" and "caca en el trabajo" in the same column
df_melted = df.melt(id_vars=['date', 'user'], value_vars=['caca', 'caca en el trabajo'], var_name='tipo', value_name='cantidad')
df_melted['date'] = pd.to_datetime(df_melted['date'])
df_melted

In [ ]:
# Group data by user and date, summing the 'cantidad' values
grouped_data = df_melted.groupby(['user', 'date'])['cantidad'].sum().reset_index()

# Get unique users
unique_users = grouped_data['user'].unique()

# Create subplots for each user
fig, axes = plt.subplots(len(unique_users), 1, figsize=(21, 4 * len(unique_users)), sharex=True)

# Set the title for the entire plot
fig.suptitle('Track Gran Cagatón Anual', fontsize=30)

# Iterate through each user and create a calendar map
for i, user in enumerate(unique_users):
    user_data = grouped_data[grouped_data['user'] == user].set_index('date')
    calmap.yearplot(user_data['cantidad'], year=user_data.index.year.unique()[0], ax=axes[i], cmap='YlOrBr', monthticks=1)
    # set fontsize for axis labels
    axes[i].set_title(f"{user}")
    axes[i].title.set_fontsize(20)

# center it
fig.tight_layout()

# save the plot in the 'plots' folder with day and month in the filename
plt.savefig(f'plots/calendar_{dateString}.png')

plt.show()
